In [7]:
import pandas as pd
mentees = pd.read_csv('~/Downloads/Cohort 3 Data - Mentees.csv')
mentors = pd.read_csv('~/Downloads/Cohort 3 Data - Mentors.csv')
#Clean DF
mentees_flitered = mentees.filter(items=["Id", 
                 "What's your email?",
                 'What role are you looking for help in?',
                 "Which best describes the position you're in?",
                 'How many years of professional experience do you have in your current role?',
                 'What is the main goal you want help with from your mentor?',
                 'How many years of experience do you have as a founder?',
                 'What industry are you looking for help with?',
                 'What company stage do you want your mentor to be in?',
                 'What timezone are you in?',
                 'Have you been matched before?',
                 "Tell us why you want to be a part of Lenny's Mentorship Program!",
                 'What is the main goal you want help with from your mentor?'
                ])

mentors_flitered = mentors.filter(items=["Id", 
                 "What's your email?",
                 'Which functional role do you want your mentee to be in?',
                 "Which best describes the position you're in?",
                 'How many years of professional experience do you have in the role you want to be a mentor for?',
                 'Which goals can you help your mentee with?',
                 'How many years of experience do you have as a founder?',
                 'What industry can you help your mentee with?',
                 'What company stage do you want your mentee to be in?',
                 'What timezone are you in?',
                 'Have you been matched before?',
                 'What goals can you help your mentee with?'
                ])
#print(mentors.columns.values)
#print(mentors_flitered.columns.values)
#display(mentees_flitered)
#display(mentors_flitered)


In [8]:
#Input comma seperated list of value
#Output list of values with whitespace stipped off
def clean_multiselect(x):
    if isinstance(x, str):
        return list(map(str.strip,x.split(',')))
    else:
        return []


In [9]:
#Input Dataframe and multi-select field to Binarize
from sklearn.preprocessing import MultiLabelBinarizer
def MultiLableBinarize_df(input_frame, collumn_name):
    nested_list = list(map(clean_multiselect,input_frame[collumn_name].to_list()))
    mlb = MultiLabelBinarizer()
    mlb_df = pd.DataFrame(mlb.fit_transform(nested_list), columns=mlb.classes_)
    bigger = pd.concat([input_frame,mlb_df],axis=1)
    return bigger


In [10]:
temp = MultiLableBinarize_df(mentees_flitered,'What industry are you looking for help with?')
display(temp)

,Id,What's your email?,What role are you looking for help in?,Which best describes the position you're in?,How many years of professional experience do you have in your current role?,What is the main goal you want help with from your mentor?,How many years of experience do you have as a founder?,What industry are you looking for help with?,What company stage do you want your mentor to be in?,What timezone are you in?,...,Medtech selling to insurance with d2c marketing,Non-profit,PropTech,Proptech,Restaurant Technology,Smart Home,Supply Chain,Travel,Web3,igaming affiliation
0,1.0,NaN,Career Pathing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2.0,NaN,Product,Individual contributor (IC),6-8,Level up in career,NaN,Health & Wellness,Series A,Central European Time (GMT +2),...,0,0,0,0,0,0,0,0,0,0
2,3.0,NaN,Founder,NaN,NaN,NaN,0-2,"DevOps / Dev tools, B2B Saas",Seed or earlier,Eastern Time (GMT -4),...,0,0,0,0,0,0,0,0,0,0
3,4.0,NaN,Product,Individual contributor (IC),0-2,Transition into product management,NaN,"B2B Saas, Fintech",Series D+,Eastern Time (GMT -4),...,0,0,0,0,0,0,0,0,0,0
4,5.0,NaN,Product,Manager of ICs,0-2,Transition into product management,NaN,B2B Saas,Series B,Central European Time (GMT +2),...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,237.0,NaN,Product,Individual contributor (IC),0-2,Become better in my IC role,NaN,"Cybersecurity, AI",Public company,Pacific Time (GMT -7),...,0,0,0,0,0,0,0,0,0,0
237,238.0,NaN,Product,Individual contributor (IC),0-2,Level up in career,NaN,"Health & Wellness, B2B Saas",Series C,Pacific Time (GMT -7),...,0,0,0,0,0,0,0,0,0,0
238,239.0,NaN,Product,Individual contributor (IC),0-2,Become better in my IC role,NaN,"B2B Saas, Web3",Series B,Eastern Time (GMT -4),...,0,0,0,0,0,0,0,0,1,0
239,240.0,NaN,Product,Manager of ICs,0-2,Become better in my management role,NaN,"Fintech, AI",Series C,Eastern Time (GMT -4),...,0,0,0,0,0,0,0,0,0,0
